In [1]:
!nvidia-smi

Thu Nov  5 18:17:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -qq wandb
!git clone https://github.com/lattice-ai/GPT
%cd GPT

     |████████████████████████████████| 1.8MB 7.9MB/s 
     |████████████████████████████████| 163kB 49.7MB/s 
     |████████████████████████████████| 102kB 15.1MB/s 
     |████████████████████████████████| 102kB 15.1MB/s 
     |████████████████████████████████| 133kB 51.3MB/s 
     |████████████████████████████████| 71kB 12.1MB/s 
Cloning into 'GPT'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (220/220), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 220 (delta 95), reused 180 (delta 55), pack-reused 0
Receiving objects: 100% (220/220), 127.84 KiB | 18.26 MiB/s, done.
Resolving deltas: 100% (95/95), done.
/content/GPT


In [3]:
from gpt.experiments.language_model import IMDBReviewLanguageExperiment

experiment = IMDBReviewLanguageExperiment()

In [4]:
from gpt.experiments.utils import init_wandb

init_wandb(
    project_name='gpt', experiment_name='imdb_language_model',
    wandb_api_key='41442f74323655b7ec081f4af030fa821f294c72'
)

wandb: Currently logged in as: lattice-ai (use `wandb login --relogin` to force relogin)


In [5]:
experiment.build_dataset(
    'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz')

experiment.compile()

84131840/84125825 [==============================] - 13s 0us/step
Dataset Size: 50000 files
Dataset: <PrefetchDataset shapes: ((None, 100), (None, 100)), types: (tf.int64, tf.int64)>
Model: "GPT_depth_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
GPT_Input (InputLayer)       [(None, 100)]             0         
_________________________________________________________________
combined_encoding (CombinedE (None, 100, 256)          5145600   
_________________________________________________________________
transformer_block (Transform (None, 100, 256)          395776    
_________________________________________________________________
GPT_Output (Dense)           (None, 100, 20000)        5140000   
Total params: 10,681,376
Trainable params: 10,681,376
Non-trainable params: 0
_________________________________________________________________


In [6]:
start_text = 'the actor was'
start_tokens = experiment.tokenize(start_text=start_text)
experiment.train(
    epochs=30, start_tokens=start_tokens,
    max_length=100, max_tokens=40, top_k=10,
    infer_every=1, log_on_wandb=True
)

Epoch 1/30
   1563/Unknown - 149s 96ms/step - loss: 5.0529 - GPT_Output_loss: 5.0529
Sample Generate Text: the actor was , was in one of the worst movies ever made by all the time , but it was one of all time . my favorite episodes i have to read the book by one of the most of the series of , was in one of the worst movies ever made by all the time , but it was one of all time . my favorite episodes i have to read the book by one of the most of the series of

Epoch 00001: saving model to wandb/run-20201105_182056-rwkad0fw/files/gpt_language_model_checkpoint1
1563/1563 [==============================] - 152s 97ms/step - loss: 5.0529 - GPT_Output_loss: 5.0529
Epoch 2/30
1563/1563 [==============================] - ETA: 0s - loss: 4.4802 - GPT_Output_loss: 4.4802
Sample Generate Text: the actor was an excellent film , in [UNK] , an actress who is a good actress . she is great and the acting is great . she also is good , but she is a lot better than that ! ! ! ! an excellent film , in [UNK

In [7]:
start_text = 'the movie is'
start_tokens = experiment.tokenize(start_text=start_text)
output = experiment.infer(
    start_tokens, max_length=100, max_tokens=40, top_k=10
)
print('Generated Text:', output)

Generated Text: the movie is an entertaining movie ! one of the acting is superb . this movie is amazing to look like an oscar nominee for a year award . he is an intelligent and talented actors . . . . . .i think i an entertaining movie ! one of the acting is superb . this movie is amazing to look like an oscar nominee for a year award . he is an intelligent and talented actors . . . . . .i think i
